# Задание 2.
## Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/). Список email-ов задаем переменной в начале кода:
```
EMAIL = [xxx@x.ru, yyy@y.com]
```
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

# Получаем данные из API

In [1]:
import requests
import json

EMAIL = ["xxx@x.ru", "yyy@y.com"]

url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
data = {"emailAddresses": EMAIL}
headers = {
    "Vaar-Version": "0",
    "Vaar-Header-App-Product": "hackcheck-web-avast",
    "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
    "Vaar-Header-App-Build-Version": "1.0.0",
}
r = requests.post(url=url, data=json.dumps(data), headers=headers)

# Формируем датафрейм по утечкам

In [2]:
import pandas as pd

def generate_breaches_dataframe(avast_response):
    avast_response_json = avast_response.json()
    
    breaches = pd.DataFrame()
    for breach in avast_response_json['breaches'].values():  
        # remove extra column
        del breach['statistics']
        # generate dataframe for one record
        df = pd.DataFrame(breach.values(), index=breach.keys())
        # transpose and accumulate it
        breaches = breaches.append(df.T)

    return breaches.reset_index(drop=True)
    
breaches = generate_breaches_dataframe(r)
breaches.head()

,breachId,site,recordsCount,description,publishDate
0,16613,verifications.io,677914246,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
1,17609,vk.com,42536910,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z
2,17110,azcentral.com,705538,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z
3,17670,wishbone.io,37475476,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z
4,13094,myheritage.com,110041653,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z


# Формируем датафрейм по ящикам

In [3]:
def generate_emails_dataframe(avast_response):
    emails = pd.DataFrame()
    for email, breaches in r.json()['summary'].items():
        # prepare list of id
        breachIds = list(breaches.values())[0]
        # generate dataframe and transpose
        df = pd.DataFrame.from_dict({'email': [email], 'breachId': breachIds}, orient='index').T
        # for every breachId set current email
        df.email.fillna(email, inplace=True)
        # accumulate dataframes
        emails = emails.append(df)
    
    return emails.reset_index(drop=True)


emails = generate_emails_dataframe(r)
emails.head()

,email,breachId
0,xxx@x.ru,16613
1,xxx@x.ru,17609
2,xxx@x.ru,3176
3,xxx@x.ru,12
4,xxx@x.ru,3


# Мержим датафреймы и выводим результат

In [4]:
merged = emails.merge(breaches, on='breachId')
merged[['email', 'breachId', 'site', 'description']]

,email,breachId,site,description
0,xxx@x.ru,16613,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,17609,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,3176,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,12,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,3,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,yyy@y.com,3,adobe.com,"In October of 2013, criminals penetrated Adobe..."
6,xxx@x.ru,3164,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
7,xxx@x.ru,2961,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
8,xxx@x.ru,15,imesh.com,"In June 2016, a cache of over 51 million user ..."
9,yyy@y.com,15,imesh.com,"In June 2016, a cache of over 51 million user ..."


## Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: [https://vk.com/dev/methods](https://vk.com/dev/methods) , вам поможет метод [wall.get](https://vk.com/dev/wall.get)
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>